# Task 3 - Generative tasks with external models

This notebook contains a prototype of a RAG system based on LLaVA model. This task will consist of enhancing users multimodal prompts by providing related information from our data pipeline. The related info will be added according to its similarity with the user request. The aim is to provide the model with better context to create the response.

In [ ]:
# Get Prompt from user

# Extract different types of data
user_prompt_text = "What is this cancer? Which is the diagnosis and how can it be treated?"
user_prompt_images = []
user_prompt_audios = []

# Retrieve similar results

In [1]:
# Perform query to LLaVA

import base64
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig

# Load the model in half-precision
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", dtype=torch.float16, device_map="auto", load_in_4bit=True)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

image = Image.open("test.jpg").convert("RGB")

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": user_prompt_text},
        ],
    },
]

inputs = processor.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, torch.float16)

# Generate
generate_ids = model.generate(**inputs, max_new_tokens=30)
processor.batch_decode(generate_ids, skip_special_tokens=True)

PackageNotFoundError: No package metadata was found for bitsandbytes